In [93]:
import pandas as pd

In [94]:
matches = pd.read_csv('serie_a_matches.csv')

In [95]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,team,season
0,2022-08-15,18:30,Serie A,Matchweek 1,Mon,Away,W,5,2,Hellas Verona,...,Match Report,NaN,25.0,8.0,15.2,0.0,0,0,Napoli,2022
1,2022-08-21,18:30,Serie A,Matchweek 2,Sun,Home,W,4,0,Monza,...,Match Report,NaN,22.0,5.0,15.3,1.0,0,0,Napoli,2022
2,2022-08-28,20:45,Serie A,Matchweek 3,Sun,Away,D,0,0,Fiorentina,...,Match Report,NaN,13.0,2.0,14.7,1.0,0,0,Napoli,2022
3,2022-08-31,20:45,Serie A,Matchweek 4,Wed,Home,D,1,1,Lecce,...,Match Report,NaN,19.0,7.0,17.7,0.0,0,0,Napoli,2022
4,2022-09-03,20:45,Serie A,Matchweek 5,Sat,Away,W,2,1,Lazio,...,Match Report,NaN,19.0,7.0,16.1,0.0,0,0,Napoli,2022


In [96]:
matches.dtypes

date              object
time              object
comp              object
round             object
day               object
venue             object
result            object
gf                 int64
ga                 int64
opponent          object
xg               float64
xga              float64
poss             float64
attendance       float64
captain           object
formation         object
opp formation     object
referee           object
match report      object
notes            float64
sh               float64
sot              float64
dist             float64
fk               float64
pk                 int64
pkatt              int64
team              object
season             int64
dtype: object

In [97]:
matches["date"] = pd.to_datetime(matches["date"])
matches["time"] = matches["time"].str.split(":").str[0]

In [98]:
matches["time"] = matches["time"].astype(int)

In [99]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes

In [100]:
matches["venue_code"].value_counts()

venue_code
0    1065
1    1065
Name: count, dtype: int64

In [101]:
matches["opp_code"] = matches["opponent"].astype("category").cat.codes

In [102]:
matches["day_code"] = matches["date"].dt.dayofweek

In [103]:
matches = matches.drop(columns=["referee", "captain", "formation", "opp formation"])

In [104]:
matches = matches.drop(columns=["match report", "notes"])

In [105]:
matches["target"] = matches["result"].map({"W": 1, "D": 0, "L": -1})

In [106]:
matches["target"].value_counts()

target
 1    906
-1    686
 0    538
Name: count, dtype: int64

In [107]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
predictor_columns = [f"{col}_rolling" for col in cols]

In [108]:
predictor_columns

['gf_rolling',
 'ga_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling']

In [109]:
#calculate avg stats of the last 3 games for each team
def roll_avg(team_group, cols, predictor_columns):
    team_group = team_group.sort_values("date")
    roll_stats = team_group[cols].rolling(window=3, closed="left").mean()
    team_group[predictor_columns] = roll_stats
    team_group = team_group.dropna(subset=predictor_columns)
    return team_group

In [110]:
matches_roll = matches.groupby("team").apply(lambda x: roll_avg(x, cols, predictor_columns))

In [111]:
matches_roll = matches_roll.droplevel("team")

In [112]:
matches_roll.columns

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'sh', 'sot', 'dist',
       'fk', 'pk', 'pkatt', 'team', 'season', 'venue_code', 'opp_code',
       'day_code', 'target', 'gf_rolling', 'ga_rolling', 'sh_rolling',
       'sot_rolling', 'dist_rolling', 'fk_rolling', 'pk_rolling',
       'pkatt_rolling'],
      dtype='object')

In [113]:
matches_roll.index = range(matches_roll.shape[0])

In [114]:
matches_roll["team"].value_counts()

team
Napoli            225
Lazio             187
Internazionale    149
Milan             149
Atalanta          149
Fiorentina        111
Roma              111
Juventus          111
HellasVerona       74
Torino             73
Sassuolo           73
Salernitana        73
Monza              73
Lecce              73
Bologna            73
Empoli             73
Udinese            73
Spezia             36
Cremonese          35
Frosinone          35
Cagliari           35
Sampdoria          35
Genoa              35
Name: count, dtype: int64

In [115]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

In [205]:
def prediction(df, predictors):
    train = df[df["date"] < "2023-01-01"]
    test = df[df["date"] >= "2023-01-01"]

    model = RandomForestClassifier(n_estimators=200, min_samples_split=5, random_state=1)

    model.fit(train[predictors], train["target"])
    predictions = model.predict(test[predictors])

    all_data = pd.DataFrame(dict(actual=test["target"], predicted=predictions))

    precision = precision_score(test["target"], predictions, average=None)
    return precision, all_data

In [206]:
pred_cols = ["venue_code", "opp_code", "time", "day_code"] + predictor_columns

In [207]:
precision, data = prediction(matches_roll, pred_cols)

In [208]:
precision

array([0.46734694, 0.34177215, 0.4908896 ])

In [209]:
data = data.merge(matches_roll[["team", "date", "opponent", "result"]], left_index=True, right_index=True)

In [210]:
data

,actual,predicted,team,date,opponent,result
42,0,1,Atalanta,2023-01-04,Spezia,D
43,0,1,Atalanta,2023-01-04,Spezia,D
44,0,1,Atalanta,2023-01-04,Spezia,D
45,1,1,Atalanta,2023-01-09,Bologna,W
46,1,1,Atalanta,2023-01-09,Bologna,W
...,...,...,...,...,...,...
2056,0,-1,Udinese,2024-04-28,Bologna,D
2057,0,-1,Udinese,2024-05-06,Napoli,D
2058,1,-1,Udinese,2024-05-13,Lecce,W
2059,0,0,Udinese,2024-05-19,Empoli,D


In [211]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {
    "Internazionale": "Inter",
    "Hellas Verona": "HellasVerona",
}

mapping = MissingDict(**map_values)

In [212]:
data["new_team"] = data["team"].map(mapping)

In [213]:
merged = data.merge(data, left_on=["new_team", "date"], right_on=["opponent", "date"])

In [214]:
merged

,actual_x,predicted_x,team_x,date,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
0,0,1,Atalanta,2023-01-04,Spezia,D,Atalanta,0,1,Spezia,Atalanta,D,Spezia
1,0,1,Atalanta,2023-01-04,Spezia,D,Atalanta,0,1,Spezia,Atalanta,D,Spezia
2,0,1,Atalanta,2023-01-04,Spezia,D,Atalanta,0,1,Spezia,Atalanta,D,Spezia
3,1,1,Atalanta,2023-01-09,Bologna,W,Atalanta,-1,-1,Bologna,Atalanta,L,Bologna
4,1,1,Atalanta,2023-01-09,Bologna,W,Atalanta,-1,-1,Bologna,Atalanta,L,Bologna
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2117,0,-1,Udinese,2024-04-28,Bologna,D,Udinese,0,1,Bologna,Udinese,D,Bologna
2118,0,-1,Udinese,2024-05-06,Napoli,D,Udinese,0,-1,Napoli,Udinese,D,Napoli
2119,1,-1,Udinese,2024-05-13,Lecce,W,Udinese,-1,1,Lecce,Udinese,L,Lecce
2120,0,0,Udinese,2024-05-19,Empoli,D,Udinese,0,0,Empoli,Udinese,D,Empoli


Actual wins predicted

In [215]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] == -1)]["actual_x"].value_counts()

actual_x
 1    309
 0     98
-1     62
Name: count, dtype: int64

In [219]:
309/(309+98+62)

0.6588486140724946

Actual draws predicted

In [216]:
merged[(merged["predicted_x"] == 0) & (merged["predicted_y"] == 0)]["actual_x"].value_counts()

actual_x
 0    6
 1    5
-1    5
Name: count, dtype: int64

In [218]:
6/16

0.375

Actual lost predicted

In [217]:
merged[(merged["predicted_x"] == -1) & (merged["predicted_y"] == 1)]["actual_x"].value_counts()

actual_x
-1    295
 0     83
 1     61
Name: count, dtype: int64

In [220]:
295/(295+83+61)

0.6719817767653758